In [47]:
#importing all the libraries we will be needing

import numpy as np
from scipy.stats import norm
from datetime import datetime as dt
import pandas as pd

In [48]:
#This function calculates the d1 and d2 which will be requried to calculate N_d1 and N_d2 and used to calculate the greeks

def d1_d2(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return d1, d2

In [49]:
#This function calculate the greeks using the function d1_d2

def greeks(S, K, r, sigma, T):
    d1, d2 = d1_d2(S, K, r, sigma, T)
    
    N_d1 = norm.pdf(d1)
    N_d2 = norm.pdf(d2)
    
    delta = norm.cdf(d1)
    gamma = N_d1 / (S * sigma * np.sqrt(T))
    vega = S * np.sqrt(T) * N_d1
    theta = -((S * N_d1 * sigma) / (2 * np.sqrt(T))) - (r * K * np.exp(-r * T) * norm.cdf(d2))
    rho = K * T * np.exp(-r * T) * norm.cdf(d2)

    return delta, gamma, vega, theta, rho

In [50]:
#This function calculates the dividend yeild 

def div_yield(C, P, S, K, r, T):
    
    K0 = K * np.exp(-r * T)
    S1 = C - P + K0
    if S1 <= 0 or S <= 0:  
        return None
    x = -np.log(S1 / S) / T
    return x

In [51]:
#All the functions that will be used has been created. We will be providing the inputs below.

S = 60.48  
r = 0.0033  
sigma = 0.20

#This data is call option data having different strike prices and maturity dates
data = [[60,"2013-02-16"],
       [60,"2013-03-16"],
       [60,"2013-06-22"],
       [65,"2013-06-22"],
       [60,"2013-09-21"],
       [65,"2013-09-21"],
       [60,"2014-01-08"],
       [65,"2014-01-18"],
       [55,"2015-01-17"],
       [65,"2015-01-17"]
      ]

columns = ["Strike Price","Maturity Date","Delta","Gamma","Vega","Theta","Rho"]
final = pd.DataFrame(columns=columns)

for option in data:
    
    K = option[0]
    maturity = dt.strptime(option[1], "%Y-%m-%d")
    today = dt.strptime("2013-02-13", "%Y-%m-%d")
    T = (maturity - today).days / 365
    delta, gamma, vega, theta, rho = greeks(S, K, r, sigma, T)
    
    option_final = pd.DataFrame({
            "Strike Price": [K],
            "Maturity Date": [option[1]],
            "Delta": [delta],
            "Gamma": [gamma],
            "Vega": [vega],
            "Theta": [theta],
            "Rho": [rho]
    })
    
    final = pd.concat([final, option_final], ignore_index=True)
    
print("The option maturing on a different dates having a strike price K has Greeks: Delta, Gamma, Vega, Theta and Rho\n")
print(final)

The option maturing on a different dates having a strike price K has Greeks: Delta, Gamma, Vega, Theta and Rho

  Strike Price Maturity Date     Delta     Gamma       Vega      Theta  \
0           60    2013-02-16  0.673651  0.328760   1.976790 -24.183019   
1           60    2013-03-16  0.567754  0.111535   6.929979  -8.267319   
2           60    2013-06-22  0.554198  0.054965  14.211426  -4.121318   
3           65    2013-06-22  0.295660  0.048031  12.418510  -3.568606   
4           60    2013-09-21  0.556367  0.042057  18.544826  -3.174494   
5           65    2013-09-21  0.354301  0.039616  17.468524  -2.962067   
6           60    2014-01-08  0.560631  0.034337  22.642205  -2.607745   
7           65    2014-01-18  0.396791  0.033071  22.470241  -2.488812   
8           55    2015-01-17  0.692824  0.020932  29.493229  -1.637618   
9           65    2015-01-17  0.460971  0.023651  33.324839  -1.805609   

         Rho  
0   0.328966  
1   2.775139  
2  10.737198  
3   5.873421 

In [58]:
#This data is the prices of the call and put options having different strike prices and maturity dates

call_put = [
    ["call", 60, "2013-02-16", 0.550],
    ["put",  60, "2013-02-16", 0.050],
    ["call", 60, "2013-03-16", 1.300],
    ["put",  60, "2013-03-16", 0.800],
    ["call", 60, "2013-06-22", 1.925],
    ["put",  60, "2013-06-22", 2.150],
    ["call", 65, "2013-06-22", 0.575],
    ["put",  65, "2013-06-22", 2.600],
    ["call", 70, "2013-09-21", 0.875],
    ["put",  70, "2013-09-21", 3.200],
    ["call", 55, "2014-01-18", 2.325],
    ["put",  55, "2014-01-18", 1.725],
    ["call", 60, "2014-01-18", 1.650],
    ["put",  60, "2014-01-18", 2.250],
    ["call", 65, "2014-01-18", 1.025],
    ["put",  65, "2014-01-18", 2.875],
    ["call", 55, "2015-01-17", 6.650],
    ["put",  55, "2015-01-17", 5.550],
    ["call", 65, "2015-01-17", 1.425],
    ["put",  65, "2015-01-17", 7.600]
]

from collections import defaultdict
option_pairs = defaultdict(dict)

#this converages the options of same strike price and maturity date
for option in call_put:
    key = (option[1], option[2])
    option_pairs[key][option[0]] = option[3]

cols = ["Strike","Maturity","Dividend"]
yields = pd.DataFrame(columns=cols)

for (strike, maturity), prices in option_pairs.items():
    if "call" in prices and "put" in prices:
        call_price = prices["call"]
        put_price = prices["put"]
        maturity = dt.strptime(option[2], "%Y-%m-%d")
        today = dt.strptime("2013-02-13", "%Y-%m-%d")
        timedelta = (maturity - today).days / 365
        
        div = div_yield(call_price, put_price, S, strike, r, timedelta)
        
        yields_final = pd.DataFrame({
            "Strike": [strike],
            "Maturity": [maturity],
            "Dividend": [div]
        })
        
        yields = pd.concat([yields, yields_final], ignore_index=True)

print(yields)

  Strike   Maturity  Dividend
0     60 2015-01-17  0.003101
1     60 2015-01-17  0.003101
2     60 2015-01-17  0.009400
3     65 2015-01-17 -0.017582
4     70 2015-01-17 -0.054947
5     55 2015-01-17  0.046945
6     60 2015-01-17  0.012689
7     65 2015-01-17 -0.019033
8     55 2015-01-17  0.042267
9     65 2015-01-17  0.018053
